In [2]:
from netCDF4 import Dataset
import numpy as np

In [3]:
data = 'data/sbmn_cappi_20140103_031208.nc'
fh= Dataset(data,mode='r')

Extracting and plotting point Data

In [4]:
fh.dimensions.keys()

odict_keys(['time', 'bounds', 'x0', 'y0', 'z0'])

In [5]:
fh.variables.keys()

odict_keys(['time', 'start_time', 'stop_time', 'time_bounds', 'x0', 'y0', 'lat0', 'lon0', 'z0', 'grid_mapping_0', 'VEL', 'DBZc', 'range', 'Coverage'])

In [6]:
fh.variables['DBZc']

<class 'netCDF4._netCDF4.Variable'>
float32 DBZc(time, z0, y0, x0)
    valid_min: -21.54068
    valid_max: 49.67983
    _FillValue: -9999.0
    standard_name: DBZc
    long_name: CZ
    units: dBZ
    coordinates: lon0 lat0
    grid_mapping: grid_mapping_0
unlimited dimensions: 
current shape = (1, 40, 241, 241)
filling on

In [7]:
lons = fh.variables['lon0'][:]
lats = fh.variables['lat0'][:]
dbz = fh.variables['DBZc'][:][0][0:240]

dbz_units = fh.variables['DBZc'].units

## Ploting georefeced data

In [8]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [9]:
lat_0 = lats.mean()
lon_0 = lons.mean()

m = Basemap(width=5000000,height=3500000,
            resolution='l',projection='stere',\
            lat_ts=40,lat_0=lat_0,lon_0=lon_0,)

/home/mastercontrol/miniconda3/envs/cap394/lib/python3.6/site-packages/ipykernel_launcher.py:6: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  


In [ ]:
# Because our lon and lat variables are 1D,
# use meshgrid to create 2D arrays
# Not necessary if coordinates are already in 2D arrays.
lon , lat = np.meshgrid(lons[0][:], lats[0][:])
xi, yi = m(lon, lat)

In [ ]:
# Plot Data
cs = m.pcolor(xi,yi,np.squeeze(dbz.data[0]))

# # Add Grid Lines
m.drawparallels(np.arange(-80., 81., 10.), labels=[1,0,0,0], fontsize=10)
m.drawmeridians(np.arange(-180., 181., 10.), labels=[0,0,0,1], fontsize=10)


# # Add Coastlines, States, and Country Boundaries
m.drawcoastlines()
m.drawstates()
m.drawcountries()

# # Add Colorbar
cbar = m.colorbar(cs, location='bottom', pad="10%", size='10%')
cbar.set_label(dbz_units)

# # Add Title
plt.title('DBZ Max')
plt.show()

In [ ]:
dbz = fh.variables['DBZc'][:][0][0:240]
dados = dbz.data

In [ ]:
count = 0
for i in dados:
    print(i)
    count = count + 1

In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature

In [ ]:
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)

In [ ]:
# Open the NetCDF file
ncfile = Dataset("data/sbmn_cappi_20140103_031208.nc")

In [ ]:
ncfile.variables.keys()